---
title: "Resampling-Methoden und Vorhersagen"
jupyter: ir
---


In [ ]:
#| echo: false
#| include: false
# Daten laden (Setup aus ML-Einführung)

req_pkg <- c(
  "riskCommunicator", "data.table", "tinyplot", "see",
  "tidymodels", "parsnip", "yardstick", "parallel",
  "ranger", "rsample", "workflows", "tune", "dials",
  "bonsai", "lightgbm", "kdry"
)
for (r in req_pkg) {
  if (!(r %in% installed.packages()[, "Package"])) {
    install.packages(r)
  }
}

dataset_full <- riskCommunicator::framingham |>
  data.table::data.table() # Daten einlesen
# Subset: Basisuntersuchung
dataset <- dataset_full[get("PERIOD") == 1, ]

# Relevante Spalten definieren
use_cols <- c("SEX", "TOTCHOL", "AGE", "SYSBP",
"CURSMOKE", "CIGPDAY", "BMI", "DIABETES",
"HYPERTEN")
# Relevante Spalten filtern, fehlende Werte entfernen
dataset <- dataset[
  , .SD, .SDcols = use_cols
] |>na.omit()

# Transformieren der katgeorialen Variablen
# "SEX" "CURSMOKE" "CIGPDAY" "DIABETES" "HYPERTEN"
cat_vars <- use_cols[c(1, 5, 8, 9)]
# Datentyp "factor" ändern
dataset[, (cat_vars) := lapply(
  X = .SD,
  FUN = factor),
  .SDcols = cat_vars
]
rm(dataset_full, cat_vars, use_cols, r, req_pkg)

# Teildatensatz für Regressions-Beispiele
# Regression: Zielvariable "SYSBP" --> Entfernen von "HYPERTEN"
dataset_reg <- dataset[
  , .SD, .SDcols = setdiff(colnames(dataset), "HYPERTEN")
]


In [ ]:
#| echo: false
#| include: false
# Parallelisieren, wenn möglich
nc <- parallel::detectCores()
# Verwende Hälfte der verfügbaren CPU threads,
# jedoch mindestens 2
ncores <- pmax(2, ceiling(nc) / 2)


In [ ]:
#| echo: true
# Verwendung des {parsnip} R-Pakets
# (aus dem {tidymodels} Framework)
# um ein lineares Regressionsmodell
# auf Grundlage der R-Basisimplementierung
# {stats::lm} zu definieren
lm_spec <- parsnip::linear_reg() |>
  parsnip::set_engine("lm") |>
  parsnip::set_mode("regression")


In [ ]:
#| echo: true
# Zusammenfassung des definierten Modells
lm_spec |> parsnip::translate()


In [ ]:
#| echo: true
# Initialisieren des (Pseudo-)
# Zufallszahlengenerators
set.seed(123)
# Erstellen eines Train-/ Test-Splits
# (60% Trainingsdaten, 40% Testdaten)
# mithilfe des {rsample} R-Pakets;
# Die Daten werden `stratifiziert`
# aufgeteilt, sodass die Zielvariable
# in beiden Teildatensätzen ähnlich
# verteilt ist (`strata="SYSBP"`)
init_split <- rsample::initial_split(
  data = dataset_reg,
  prop = 0.6,
  strata = "SYSBP"
)
init_split |> print()


In [ ]:
#| echo: true
# Zielvariable: Trainingsdaten
# Mittelwert + Standardabweichung
rsample::training(init_split)$SYSBP |>
  kdry::rep_mean_sd()
# Zielvariable: Testdaten
# Mittelwert + Standardabweichung
rsample::testing(init_split)$SYSBP |>
  kdry::rep_mean_sd()


In [ ]:
#| echo: true
# Training des Modells
lm_m1 <- lm_spec |>
  parsnip::fit(
    SYSBP ~ .,
    data = rsample::training(init_split)
)

# Ausgabe der Koeffizienten des
# linearen Regressionsmodells
lm_m1 |>
  parsnip::extract_fit_engine() |>
  coef()


In [ ]:
#| echo: true
# Vorhersage der Zielvariable der Testdaten
lm_m1_preds <- lm_m1 |>
  parsnip::augment(
    new_data = rsample::testing(init_split))

# Auf den Testdaten berechneter
# Vorhersagefehler
(lm_m1_rmse <- lm_m1_preds |>
  yardstick::rmse(
    truth = SYSBP,
    estimate = .pred
  ))


In [ ]:
#| echo: true
# Initialisieren des (Pseudo-) Zufallszahlen-
# generators mit einem anderen Startwert
set.seed(1234)
# Erstellen eines Train-/ Test-Splits
# (60% Trainingsdaten, 40% Testdaten)
init_split <- rsample::initial_split(
  data = dataset_reg,
  prop = 0.6,
  strata = "SYSBP"
)
# Inspektion des `rsplit`-Objekts
init_split |> print()


In [ ]:
#| echo: true
# Zielvariable: Trainingsdaten
# Mittelwert + Standardabweichung
rsample::training(init_split)$SYSBP |>
  kdry::rep_mean_sd()
# Zielvariable: Testdaten
# Mittelwert + Standardabweichung
rsample::testing(init_split)$SYSBP |>
  kdry::rep_mean_sd()


In [ ]:
#| echo: true
# Training des Modells
lm_m2 <- lm_spec |>
  parsnip::fit(
    SYSBP ~ .,
    data = rsample::training(init_split)
  )

# Ausgabe der Koeffizienten des
# linearen Regressionsmodells
lm_m2 |>
  parsnip::extract_fit_engine() |>
  coef()


In [ ]:
#| echo: true
# Vorhersagefehler auf den im
# Modelltraining ausgesparten
# Testdaten
lm_m2_preds <- lm_m1 |>
  parsnip::augment(
    new_data = rsample::testing(init_split)
  )

# Auf den Testdaten berechneter
# Vorhersagefehler
(lm_m2_rmse <- lm_m2_preds |>
  yardstick::rmse(
    truth = SYSBP,
    estimate = .pred
  ))


In [ ]:
#| echo: true
# Erstellen eines Trainings- (60%),
# Validierungs- (20%) und Test-Splits (20%)
set.seed(123)
init_val_split <- rsample::initial_validation_split(
  data = dataset_reg,
  prop = c(0.6, 0.2),
  strata = "SYSBP",
  breaks = 4,
  pool = 0.1
)
# Inspektion des `rsplit`-Objekts
init_val_split |> print()


In [ ]:
#| echo: true
# Training des Modells
lm_m3 <- lm_spec |>
  parsnip::fit(
    SYSBP ~ .,
    data = rsample::training(init_val_split)
  )

# Ausgabe der Koeffizienten des
# linearen Regressionsmodells
lm_m3 |>
  parsnip::extract_fit_engine() |>
  coef()


In [ ]:
#| echo: true
# Vorhersage der Zielvariable der
# Validierungsdaten
lm_m3_preds_val <- lm_m3 |>
  parsnip::augment(
    new_data = rsample::validation(
      init_val_split
  ))

# Auf den Validierungsdaten berechneter
# Vorhersagefehler
lm_m3_preds_val |>
  yardstick::rmse(
    truth = SYSBP,
    estimate = .pred
  )


In [ ]:
#| echo: true
# Vorhersage der Zielvariable der Testdaten
lm_m3_preds_test <- lm_m3 |> parsnip::augment(
  new_data = rsample::testing(
    init_val_split
))

# Auf den Testdaten berechneter Vorhersagefehler
(lm_m3_rmse <- lm_m3_preds_test |> yardstick::rmse(
  truth = SYSBP,
  estimate = .pred
))


In [ ]:
#| echo: true
# Definition der LOOCV
loocv_folds <- rsample::loo_cv(
  data = rsample::training(init_split)
)
loocv_folds |> print(n = 6)

# Erstellen eines Workflows mit dem
# besten Hyperparameter-Setting
lm_m3_wf <- workflows::workflow() |>
  workflows::add_model(lm_spec) |>
  workflows::add_formula(SYSBP ~ .)


In [ ]:
#| echo: true
# Definition einer Funktion, welche anschließend
# mit der via `lapply` definierten For-Schleife
# verwendet werden kann
fitting_function <- function(i) {
  # Speichern der aktuellen Zeile des `rsplit`-Objekts
  spr <- rsample::get_rsplit(x = loocv_folds, index = i)
  lm_m3_i <- lm_m3_wf |> # fit workflow
    parsnip::fit(data = rsample::training(spr))
  lm_m3_i |> # predict test observation, return rmse
    parsnip::augment(new_data = rsample::testing(spr)) |>
    yardstick::rmse(truth = SYSBP, estimate = .pred)
}

# Ausführen der LOOCV via `lapply` For-Schleife
loocv_results <- lapply(
  X = seq_len(nrow(loocv_folds)), FUN = fitting_function
)

# Zusammenfassen aller Einzelergebnisse in einer Tabelle
loocv_res <- do.call(rbind, loocv_results)
# Berechnung des Mittelwerts über alle Resampling-Ergebnisse
loocv_res[[".estimate"]] |> mean()


In [ ]:
#| echo: true
set.seed(123)
# 3-fach wiederholte Kreuzvalidierung
cv_folds <- rsample::vfold_cv(
  data = rsample::training(init_split),
  v = 3,
  strata = "SYSBP"
)
cv_folds |> print()


In [ ]:
#| echo: true
# Erstellen eines Workflow-Objekts
lm_m4_wf <- workflows::workflow() |>
  workflows::add_model(lm_spec) |>
  workflows::add_formula(SYSBP ~ .)

# Training des Modells mit der zuvor definierten
# Resampling-Strategie
set.seed(123)
lm_m4 <- lm_m4_wf |> tune::fit_resamples(cv_folds)

# Mittels 3-facher CV auf Trainingsdaten ermittelter
# Schätzer für Vorhersagefehler.
# (Zum Vergleich:das mit allen Trainingsdaten
# trainierte Modell `lm_m2` hat auf den Testdaten
# einen RMSE = 19.6 ergeben)
lm_m4 |> tune::collect_metrics()


In [ ]:
#| echo: true
set.seed(123)
# 10-fach wiederholte 10-fache CV
rep_cv_folds <- rsample::vfold_cv(
  data = rsample::training(init_split),
  v = 10,
  repeats = 10,
  strata = "SYSBP"
)
rep_cv_folds |> print(n=5)


In [ ]:
#| echo: true
# Training des Modells mit der zuvor definierten
# Resampling-Strategie
set.seed(123)
lm_m5 <- lm_m4_wf |> tune::fit_resamples(rep_cv_folds)

# Mittels 10-fach wiederholter 10-facher CV auf Trainings-
# daten ermittelter Schätzer für Vorhersagefehler.
# (Zum Vergleich: das mit allen Trainingsdaten
# trainierte Modell `lm_m2` hat auf den Testdaten
# einen RMSE = 19.6 ergeben)
lm_m5 |> tune::collect_metrics()


In [ ]:
#| echo: true
set.seed(123)
# Verschachtelte CV:
# innen: 5-fache CV; außen: 3-fache CV
nested_cv_folds_cv <- rsample::nested_cv(
  data = rsample::training(init_split),
  outside = rsample::vfold_cv(
    v = 3, strata = "SYSBP"
  ),
  inside = rsample::vfold_cv(
    v = 5, strata = "SYSBP"
  )
)
nested_cv_folds_cv |> print()


In [ ]:
#| echo: true
set.seed(123)
# Leere Liste zum Sammeln der Ergebnisse
nestedcv_results_cv <- list()

# For-Schleife über die einzelnen CV-Folds
set.seed(123)
for (i in seq_len(nrow(nested_cv_folds_cv))) {
  # Speichern der aktuellen Fold-Aufteilung in `current_fold`
  current_fold <- nested_cv_folds_cv[i, ]
  lm_m6_i <- lm_m4_wf |> # Durchführen des Bootstrappings
    tune::fit_resamples(current_fold[["inner_resamples"]][[1]])
  rmse_i <- lm_m6_i |> # Ermitteln der Performance-Metrik
    tune::collect_metrics()
  nestedcv_results_cv <- c(  # Abspeichern der ermittelten
    nestedcv_results_cv,     # Metrik in Ergebnis-Liste
    list(rmse_i[1, ])
  )
}

# Zusammenfassen aller Einzelergebnisse in einer Tabelle
nestedcv_res_cv <- do.call(rbind, nestedcv_results_cv)

# Berechnung des Mittelwerts über alle Resampling-Ergebnisse
nestedcv_res_cv[["mean"]] |> mean()


In [ ]:
#| echo: true
set.seed(123)
# Verschachtelte CV:
# innen: Bootstrap CV; außen: 3-fache CV
nested_cv_folds_btstrp <- rsample::nested_cv(
  data = rsample::training(init_split),
  outside = rsample::vfold_cv(
    v = 3,
    strata = "SYSBP"
  ),
  inside = rsample::bootstraps(
    times = 25,
    strata = "SYSBP"
  )
)
nested_cv_folds_btstrp |> print()


In [ ]:
#| echo: true
# Leere Liste zum Sammeln der Ergebnisse
nestedcv_results_btstrp <- list()
# For-Schleife über die einzelnen CV-Folds
set.seed(123)
for (i in seq_len(nrow(nested_cv_folds_btstrp))) {
  # Speichern der aktuellen Fold-Aufteilung in `current_fold`
  current_fold <- nested_cv_folds_btstrp[i, ]
  lm_m7_i <- lm_m4_wf |> # Durchführen des Bootstrappings
    tune::fit_resamples(current_fold[["inner_resamples"]][[1]])
  rmse_i <- lm_m7_i |> # Ermitteln der Performance-Metrik
    tune::collect_metrics()
  nestedcv_results_btstrp <- c( # Abspeichern der ermittelten
    nestedcv_results_btstrp,    # Metrik in Ergebnis-Liste
    list(rmse_i[1, ])
  )
}
# Zusammenfassen aller Einzelergebnisse in einer Tabelle
nestedcv_res_btstrp <- do.call(rbind, nestedcv_results_btstrp)
# Berechnung des Mittelwerts über alle Resampling-Ergebnisse
nestedcv_res_btstrp[["mean"]] |> mean()


In [ ]:
#| echo: true
# Bootstrapping mit Ziehen von
# 100 voneinander unabhängigen Stichproben
set.seed(123)
btstrp <- rsample::bootstraps(
  data = rsample::training(init_split),
  times = 100,
  strata = "SYSBP"
)
btstrp |> print(n = 6)


In [ ]:
#| echo: true
set.seed(123)
# Training des Modells mit der zuvor definierten
# Resampling-Strategie
lm_m8 <- lm_m4_wf |> tune::fit_resamples(btstrp)

# Mittels Bootstrapping (100 Stichproben) auf Trainings-
# daten ermittelter Schätzer für Vorhersagefehler.
# (Zum Vergleich: das mit allen Trainingsdaten
# trainierte Modell `lm_m2` hat auf den Testdaten
# einen RMSE = 19.6 ergeben)
lm_m8 |> tune::collect_metrics()


In [ ]:
#| echo: true
library(bonsai)
# Definition eines `lightgbm`-Gardient-Boosters
# für eine Regressionsaufgabe und das Tuning
# von 4 Parametern
lgb_tune_spec <- parsnip::boost_tree(
  trees = tune::tune(),
  mtry = tune::tune(),
  learn_rate = tune::tune(),
  tree_depth = tune::tune()
) |> parsnip::set_engine("lightgbm") |>
  parsnip::set_args(num_threads = ncores) |>
  parsnip::set_mode("regression")
# Ermitteln der zu optimierenden Parameter
# und Speichern in separatem Objekt
tune_params <- lgb_tune_spec |>
  parsnip::extract_parameter_set_dials()
tune_params |> print()


In [ ]:
#| echo: true
# Vervollständigung der zu optimierenden
# Parameter
tune_params_fin <- dials::finalize(
  object = tune_params,
  x = dataset_reg
)
# Je zwei Einstellungsmöglichkeiten
# pro Hyperparameter (2^4 = 16 Komb.)
tune_grid <- tune_params_fin |>
  dials::grid_regular(levels = 2)


In [ ]:
#| echo: true
#| warning: false
# nrow(tune_grid) = 16
tune_grid |> print(n = 6)


In [ ]:
#| echo: true
#| warning: false
# Erstellen eines Workflow-Objekts
lgb_wf <- workflows::workflow() |>
  workflows::add_model(lgb_tune_spec) |>
  workflows::add_formula(SYSBP ~ .)
# Verwendung des RMSE als
# Performance-Metrik
metr <- yardstick::metric_set(
  yardstick::rmse
)

# Jede Hyperparameter-Einstellung wird
# mit einer eine 3-fachen CV getestet
set.seed(123)
lgb_m1_tune_grid <- lgb_wf |>
  tune::tune_grid(
    resamples = cv_folds,
    grid = tune_grid,
    metrics = metr
  )


In [ ]:
#| echo: true
# Für unterschiedliche Hyperparameter- Einstellungen jeweils
# ermittelte Schätzer den für Vorhersagefehler.
lgb_m1_metrics <- lgb_m1_tune_grid |> tune::collect_metrics()
lgb_m1_metrics |> print(n = 6)


In [ ]:
#| echo: true
# Hyperparameter-Einstellungen der 5 besten Modelle
lgb_m1_tune_grid |> tune::show_best(metric = "rmse")

# Auswahl des besten Hyperparameter-Settings
lgb_m1_tune_grid_best <- lgb_m1_tune_grid |>
  tune::select_best(metric = "rmse")
# Die beste Hyperparameter-Einstellung
lgb_m1_tune_grid_best


In [ ]:
#| echo: true
#| warning: false
# Erstellen eines Workflows mit dem
# besten Hyperparameter-Setting
lgb_m1_final <- lgb_wf |>
  tune::finalize_workflow(
    lgb_m1_tune_grid_best
  )

# Training eines finalen Modells mit
# der besten Hyperparameter-Einstellung
# und automatische Evaluation auf den
# ausgesparten Testdaten
# (siehe auch `?tune::last_fit`)
lgb_m1_final_fit <- lgb_m1_final |>
  tune::last_fit(init_split)
# Schätzer für Vorhersagefehler auf den
# Testdaten für ausgewählte Hyperparameter-
# Einstellung
lgb_m1_final_fit |>
  tune::collect_metrics()


In [ ]:
#| echo: true
tune_params_fin <- dials::finalize(
  object = tune_params,
  x = dataset_reg
)

# Zufällige Auswahl von 20 Hyperparameter-
# Einstellungen
tune_grid_random <- tune_params_fin |>
  dials::grid_random(size = 20)

# nrow(tune_grid_random) = 20
tune_grid_random |> print(n = 6)


In [ ]:
#| echo: true
#| warning: false
# Jede Hyperparameter-Einstellung wird mit einer 3-fachen CV getestet
set.seed(123)
lgb_m2_tune_grid_rand <- lgb_wf |>
  tune::tune_grid(
    resamples = cv_folds,
    grid = tune_grid_random,
    metrics = metr
  )

# Für unterschiedliche Hyperparameter-Einstellungen jeweils ermittelte
# Schätzer den für Vorhersagefehler.
lgb_m2_metrics <- lgb_m2_tune_grid_rand |> tune::collect_metrics()
lgb_m2_metrics |> print(n = 6)


In [ ]:
#| echo: true
# Hyperparameter-Einstellungen der 5 besten Modelle
lgb_m2_tune_grid_rand |> tune::show_best(metric = "rmse")

# Auswahl des besten Hyperparameter-Settings
lgb_m2_tune_grid_rand_best <- lgb_m2_tune_grid_rand |>
  tune::select_best(metric = "rmse")

# Die beste Hyperparameter-Einstellung
lgb_m2_tune_grid_rand_best


In [ ]:
#| echo: true
#| warning: false
# Erstellen eines Workflows mit dem
# besten Hyperparameter-Setting
lgb_m2_final <- lgb_wf |>
  tune::finalize_workflow(
    lgb_m2_tune_grid_rand_best
  )

# Training eines finalen Modells mit
# der besten Hyperparameter-Einstellung
# und automatische Evaluation auf den
# ausgesparten Testdaten
lgb_m2_final_fit <- lgb_m2_final |>
  tune::last_fit(init_split)

# Vorhersagefehler auf den Testdaten
lgb_m2_final_fit |>
  tune::collect_metrics()


In [ ]:
#| echo: true
#| warning: false
# Konfiguration der Bayes'schen Optimierung
ctrlb <- tune::control_bayes(verbose = TRUE)
set.seed(123)
# Über das Argument `initial` kann das Ergebnis
# der zuvor durchgeführten Grid-Search
# übergeben werden (Vorinformation, die eine
# informierte Wahl der a-priori
# Wahrscheinlichkeit ermöglicht)
lgb_m3_tune_bayes <- lgb_wf |>
  tune::tune_bayes(
    resamples = cv_folds,
    metrics = metr,
    initial = lgb_m2_tune_grid_rand,
    param_info = tune_params_fin,
    iter = 10, # Anzahl an Such-Iterationen
    control = ctrlb
  )
lgb_m3_metrics <- lgb_m3_tune_bayes |>
  tune::collect_metrics()


In [ ]:
#| echo: true
# Die ersten 10 Ergebnisse der Bayes'schen Optimierung
lgb_m3_metrics |> print(n = 10)


In [ ]:
#| echo: true
# Hyperparameter-Einstellungen der 5 besten Modelle
lgb_m3_tune_bayes |> tune::show_best(metric = "rmse")

# Auswahl des besten Hyperparameter-Settings
lgb_m3_tune_bayes_best <- lgb_m3_tune_bayes |>
  tune::select_best(metric = "rmse")

# Die beste Hyperparameter-Einstellung
lgb_m3_tune_bayes_best


In [ ]:
#| echo: true
#| warning: false
# Erstellen eines Workflows mit dem
# besten Hyperparameter-Setting
lgb_m3_final <- lgb_wf |>
  tune::finalize_workflow(
    lgb_m3_tune_bayes_best
  )

# Training eines finalen Modells mit
# der besten Hyperparameter-Einstellung
# und automatische Evaluation auf den
# ausgesparten Testdaten
lgb_m3_final_fit <- lgb_m3_final |>
  tune::last_fit(init_split)
# Vorhersagefehler auf den Testdaten
lgb_m3_final_fit |>
  tune::collect_metrics()